# 统计图像信息

## 1. 导入工具包

In [1]:
# 导入操作系统库以获取平台信息以及对目录操作的权限
import os

# 导入数据分析库以支持结构化数据分析
import pandas

# 导入OpenCV2库以支持图像处理操作
import cv2

# 导入计算扩展库以支持数组运算
import numpy

# 导入进度条库的模块以令进程可视化
from tqdm import tqdm

# 导入科学计算库的统计函数模块以运算核密度
from scipy import stats
                                                                                                                                             
# 导入数据图形化库的图表绘制模块以令数据可视化
import matplotlib.pyplot as plt
# 将matplotlib的图表直接嵌入到Notebook之中（代替plt.show()）
%matplotlib inline

# 导入数据图形化库的颜色规格转换模块的值标准化方法以将给定值归一化为对数刻度的0-1范围（通过取数据的以10为底的对数来绘制数据）
from matplotlib.colors import LogNorm

## 2. 创建图像数据表

In [ ]:
'''
pandas的DataFrame是一个二维的数组结构，类似二维数组。输出结果为表格，包含行（rows）和列（columns）。构造方法如下：
pandas.DataFrame(data, index, columns, dtype, copy)
data : 一组数据(ndarray、series, map, lists, dict等类型)
index : 索引值，或者可以称为行标签
columns : 列标签，默认为 RangeIndex (0, 1, 2, …, n)
dtype : 数据类型
copy : 拷贝数据，默认为 False

pandas的DataFrame的append方法用途为：在表尾中添加新行，并且返回添加后的数据对象，如果添加的行中存在原数据中没有的列，那么将给原数据添加一个新列，并用nan补值。添加方法如下：
pandas.DataFrame.append(*other*, *ignore_index=False*, *verify_integrity=False*, *sort=None*)
other : 要添加的数据，可以是dataframe，dict，Seris，list等类型
ignore_index : 当参数为True时，将在数据合并后按照0，1，2，3....的顺序重新设置索引（忽略旧索引）
verify_integrity : 参数为True时，若合并的数据与原数据包含索引相同的行就会报错
'''
'''
关于路径中的目录表示：
'/' : 根目录
'./' : 当前目录
'../' : 上层目录
'../..' : 上上层目录
'''

# 创建数据表对象
DF = pandas.DataFrame()

# 指定工作目录
WorkDir = 'DataSet' #WorkDir = os.getcwd() # 指定根目录为工作目录

# 改变当前的工作目录到指定工作目录
os.chdir(WorkDir) # 所有情况下都将返回None（不返回任何值），但如果没找到指定路径则会抛出FileNotFoundError
#print("目录修改成功 %s" % os.getcwd()) # 查看修改后的工作目录（包含根目录）

for Subfolder in tqdm(os.listdir()): # 遍历工作目录下包含的子文件夹并让进度可视化

    # 改变当前的工作目录到子文件夹目录 
    os.chdir(Subfolder)

    for File in os.listdir(): # 遍历路径下包含的文件
        try: # 尝试读取图像文件并将可读取图像的信息添加到数据表
            Image = cv2.imread(File, cv2.IMREAD_COLOR) # 以默认参数（彩色图片，忽略alpha通道）读入路径对应的图片
            DF = DF.append({'类别': Subfolder, '文件名': File, '图像宽度': Image.shape[1], '图像高度': Image.shape[0]}, ignore_index = True)
        except:
            print(os.path.join(Subfolder, File), '无法读取，未能添加')

    # 改变当前的工作目录到上层目录
    os.chdir('../') # 如果不改到subfolder的上层目录（work_dir）就会在第二次循环中找不到subfolder的相对路径，除非使用绝对路径

# 改变当前的工作目录到上层目录
os.chdir('../') # 如果不改到work_dir的上层目录就会在第二次运行时找不到workdir的相对路径，除非使用绝对路径

# 输出数据表
DF

## 3. 创建图像尺寸数据分布图

In [ ]:
'''
核密度估计是一种以非参数方式估计随机变量的概率密度函数(PDF)的方法，形式如下：
\hat{f}_h(x)=\frac{1}{n}\sum_{i=1}^nK_h(x-x_i)=\frac{1}{nh}\sum_{i=1}^nK\left( \frac{x-x_i}{h} \right)
K(x)是核函数（非负，积分为1，均值为0，符合概率密度的性质）
h是带宽（h>0）
PS：由于高斯内核方便的数学性质，也经常使用K(x)=ϕ(x)，ϕ(x)为标准正态概率密度函数。

对于自由参数h的选择可以使用最小化L2风险函数（mean intergrated squared error）：
MISE(h)=E\left[ \int (\hat{f}_h(x)-f(x))^2dx \right]
在数据可视化的相关领域中，带宽的大小决定了核密度估计函数（KDE）的平滑（smooth）程度————带宽越小越undersmooth，带宽越大越oversmooth。
在POI兴趣点推荐领域中，带宽的设置与分析尺度有关————较小的带宽会使密度分布结果中出现较多的高/低值区域以揭示密度分布的局部特征；较大的带宽可以在全局尺度下使热点区域体现得更加明显。

自适应带宽的核密度估计方法是在固定带宽核密度函数的基础上通过修正带宽参数为而得到的，形式如下：
形式一
k(x)=\frac{1}{M}\sum_{j-1}^M \frac{1}{(\omega h_j)^n}K\left( \frac{x-x^{(j)}}{\omega h_j} \right)
k(x)是带宽为hj的核密度估计函数
M是样例的个数
PS ：每一个点j都有一个带宽hj，因此这叫自适应或可变
形式二
h_j=\left \{ \frac{\left [ \prod_{k=1}^{M}f(x^{(k)}) \right ]^{\frac{1}{M}}}{f(x^{(j)})}\right \}^\alpha
α为灵敏因子（0≤α≤1且通常取0.5，当α=0时自适应带宽的核密度估计就变成了固定带宽的核密度估计）
ω为带宽的参数
'''
'''
Source: https://github.com/scipy/scipy/blob/v1.9.3/scipy/stats/_kde.py#L41-L638
使用高斯核（gaussian_kde）表示核密度（kernel-density）估计，构造方法如下:
scipy.stats.gaussian_kde(dataset, bw_method=None, weights=None)
dataset ：要估计的数据点。在单变量数据的情况下，这是一个一维数组，否则是一个具有形状(# of dims，# of data)的二维数组。
bw_method ：用于计算估计器带宽的方法。这可以是'scott', 'silverman'、标量常量或可调用对象。如果是标量，这将直接用作kde.factor.如果是可调用的，它应该需要一个scipy.stats.gaussian_kde实例作为唯一参数并返回一个标量。如果无(默认)，则使用‘scott’。。
weights ：数据点的权重。这必须与数据集的形状相同。如果 None (默认)，则假定样本的权重相同
PS：适用于单元（uni-variate）和多元（multi-variate）数据。它包括自适应带宽确定。

numpy.argsort用于将数组沿着指定轴排序并返回其索引组成的数组，构造方法如下:
numpy.argsort(arr, axis=-1, kind=None, order=None)
arr : 被排序的数组
axis : 指定沿着哪个轴排序(默认值是-1即最后一维)。若为None，则会将数组拉伸为一维
kind : 排序算法(默认为'quicksort')。注意：'stable'和'mergesort'的后端都是使用timsort算法，并且在通常情况下，算法的计算结果会随数值类型的不同而发生改变（保留'mergesort'参数是为了向后兼容）
order : 当数组arr定义了字段时，此参数指定比较字段的顺序。

matplotlib.pyplot.figure用于调用画图板，构造方法如下：
matplotlib.pyplot.figure(num=None, figsize=None, dpi=None, facecolor=None, edgecolor=None, frameon=True)
num : 图像编号或名称。数字为编号，字符串为名称
figsize : 指定figure的宽和高。单位为英寸
dpi : 指定绘图对象的分辨率。即每英寸多少个像素（缺省值为80）
facecolor : 背景颜色
edgecolor : 边框颜色
frameon : 是否显示边框

matplotlib.pyplot.scatter用于绘制标记（散）点图，构造方法如下：
matplotlib.pyplot.scatter(x, y, s=None, c=None, marker=None, cmap=None, norm=None, vmin=None, vmax=None, alpha=None, linewidths=None, edgecolors=None)
x, y : 标记点的坐标
s : 标记点的面积
c : 标记点的颜色（默认值为蓝色('b')，其余颜色同plt.plot()）
marker : 标记点的样式（默认值为实心圆('o')，其余样式同plt.plot()）
cmap : 即matplotlib.colors.Colormap，相当于多个调色盘的合集
norm, vmin, vmax : 标记点的颜色亮度设置
alpha : 标记点的透明度（取值为[0, 1]，0表示完全透明，1则表示完全不透明）
linewidths : 标记点的边缘线宽
edgecolors : 标记点的边缘颜色

matplotlib.pyplot.savefig用于保存创建的图形，构造方法如下：
matplotlib.pyplot.savefig(fname, dpi=None, papertype=None, format=None, facecolor='w', edgecolor='w', transparent=False, bbox_inches=None, pad_inches=0.1, orientation='portrait')
fname : 图片的文件名是.png，pdf格式的文件是.pdf（也可以在此处指定文件位置）
dpi	: 每英寸的点数（图像质量）
papertype : 纸张类型（可以是'a0'至'a10'，'executive'，'b0'至'b10'，'letter'，'legal'，'ledger'）
format : 文件格式（例如.png，.pdf）
facecolor, edgecolor : 面色和边色（默认为白色）
bbox_inches	: 去除空白区域（将其设置为“tight”可以正确保存所保存的图形）
pad_inches : 图片周围填充大小（即进一步去除空白区域）
transparent : 图片背景透明度
Orientation : 旋转至横向或纵向
'''

# 创建数据表中图像尺寸数据的横、纵坐标数组
X = DF['图像宽度']
Y = DF['图像高度']

# 取得坐标中的最大值（作为坐标轴的范围上限）
Axis_Max = max(max(X), max(Y))

# 计算点密度
XY = numpy.vstack([X, Y]) # 按行数依次垂直堆放两个维度的数组
Kernel_Destiny = stats.gaussian_kde(XY)(XY) # 创建（高斯）核密度对象并通过得到的二维数组样本数据初始化属性

# 按密度对点进行排序，使最密集的点最后被绘制出来
idx = Kernel_Destiny.argsort() # 通过核密度对象调用数组排序方法并将（从小到大的）索引值返回给变量
X, Y, Kernel_Destiny = X[idx], Y[idx], Kernel_Destiny[idx] # 根据索引值将重新排序后的坐标以及核密度数组赋值给原数组

# 指定画图板的figsize（输出图片大小）为900*900像素
plt.figure(figsize = (9,9))

# 绘制散点图
plt.scatter(X, Y, s = 5, c = Kernel_Destiny, cmap = 'Spectral_r') # 指定横纵坐标、面积、标记点颜色、调色盘（在参数cmap的配色方案名称后部添加 _r 以颠倒颜色顺序）

'''
plt.colorbar()
plt.xticks([])
plt.yticks([])
'''

# 设置刻度线标签的字体大小
plt.tick_params(labelsize = 15) # 也可以用'medium', 'large'等字符串来表示

# 设定横、纵坐标轴的范围
plt.xlim(xmin = 0, xmax = Axis_Max)
plt.ylim(ymin = 0, ymax = Axis_Max)

# 设置横、纵坐标轴标签的名称和字体大小
plt.ylabel('height', fontsize = 25)
plt.xlabel('width', fontsize = 25)

# 保存创建的分布图
plt.savefig('数据集图像尺寸数据分布图.pdf', dpi = 120, bbox_inches = 'tight')

# 输出创建的分布图
plt.show()